## Getting Started with HTM

In [1]:
# Import libraries and packages
import numpy as np
from htm.bindings.sdr import SDR
from htm.algorithms import TemporalMemory as TM
import numpy as np
from time import sleep
from console import fg, bg, utils

ModuleNotFoundError: No module named 'htm'

In [4]:
def formatSDR(sdr):
    result = ''
    for i in range(sdr.size):
        if i > 0 and i % 8 == 0:
            result += ' '
        result += str(sdr.dense.flatten()[i])
    return result

arraySize = 80
cycleArray = np.arange(0, 10, 1)
inputSDR = SDR( arraySize )

tm = TM(columnDimensions    = (inputSDR.size,),
        cellsPerColumn      = 1,
        minThreshold        = 4,
        activationThreshold = 8,
        initialPermanence   = 0.5
        )

for cycle in range(2):
    for sensorValue in cycleArray:
        sensorValueBits = inputSDR.dense
        sensorValueBits = np.zeros(arraySize)
        sensorValueBits[sensorValue * 8 : sensorValue * 8 + 8] = 1
        inputSDR.dense  = sensorValueBits

        tm.compute(inputSDR, learn = True)
        print(format(sensorValue, '>2') + '/' + format(cycle, '1d') + ' |', formatSDR(tm.getActiveCells()), 'Active')
        print('V:' + format(sensorValue, '>2') + ' |', formatSDR(tm.getActiveCells()), 'Active')

        tm.activateDendrites(True)
        print(format(tm.anomaly, '.2f') + ' |', formatSDR(tm.getPredictiveCells()), 'Predicted')

 0/0 | 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
V: 0 | 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Predicted
 1/0 | 00000000 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
V: 1 | 00000000 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Predicted
 2/0 | 00000000 00000000 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
V: 2 | 00000000 00000000 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Predicted
 3/0 | 00000000 00000000 00000000 11111111 00000000 000

## Example - Single Order Sequence Memory

In [6]:
def formatSDR(sdr):
    result = ''
    for i in range(sdr.size):
        if i > 0 and i % 8 == 0:
            result += ' '
        result += str(sdr.dense.flatten()[i])
    return result

arraySize = 80
cycleArray = np.append( np.arange(0, 10, 1), np.arange(8, -1, -1) )
# print(cycleArray)
inputSDR = SDR( arraySize )

tm = TM(columnDimensions    = (inputSDR.size,),
        cellsPerColumn      = 1,
        minThreshold        = 1,
        activationThreshold = 2,
        initialPermanence   = 0.5
        )

for cycle in range(5):
    for sensorValue in cycleArray:
        sensorValueBits = inputSDR.dense
        sensorValueBits = np.zeros(arraySize)
        sensorValueBits[sensorValue * 8:sensorValue * 8 + 8] = 1
        inputSDR.dense = sensorValueBits

        tm.compute(inputSDR, learn=True)
        print('V:' + format(sensorValue, '>2') + ' |', formatSDR(tm.getActiveCells()), 'Active')

        tm.activateDendrites(True)
        print(format(tm.anomaly, '.2f') + ' |', formatSDR(tm.getPredictiveCells()), 'Predicted')

V: 0 | 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Predicted
V: 1 | 00000000 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Predicted
V: 2 | 00000000 00000000 11111111 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Predicted
V: 3 | 00000000 00000000 00000000 11111111 00000000 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000 Predicted
V: 4 | 00000000 00000000 00000000 00000000 11111111 00000000 00000000 00000000 00000000 00000000 Active
1.00 | 00000000 00000000 00000000 00000000 00000000 

## Example - Simple console

In [10]:
def formatCell(cellName, activeState, winnerState, predictedState):
    styleFg = fg.white
    styleBg = bg.black
    style = None
    if(activeState == 1):
        styleFg = fg.green
    if(winnerState == 1):
        styleBg = bg.i22
    if(predictedState == 1):
        styleBg = bg.i241
    
    style = styleFg + styleBg
    if(style != None):
        result = style(format(cellName,'2d'))
    else:
        result = format(cellName,'2d')
    return result

def printHeader(step, sensorValue):
    print('-' * dashMultiplyer)
    print('| Cycle', format(cycle+1,'2d'), '| Step', format(step,'3d'), '| Value', format(sensorValue,'3d'),  '| Anomaly:', format(tm.anomaly, '.1f'), '|')
    print('-' * dashMultiplyer)
    colHeader = '| Column | '
    for colIdx in range(columns):
        colHeader +=  format(colIdx,'2d') + ' | '
    print(colHeader)
    print('-' * dashMultiplyer)

def printConnectionDetails(tm):
    for cell in range(columns * cellsPerColumn):
        segments = tm.connections.segmentsForCell(cell)
        for segment in segments:
            num_synapses = tm.connections.numSynapses(segment)
            for synapse in tm.connections.synapsesForSegment(segment):
                presynCell = tm.connections.presynapticCellForSynapse(synapse)                    
                permanence = tm.connections.permanenceForSynapse(synapse)
                print('cell', format(cell,'2d'), 'segment', format(segment,'2d'), 'has synapse to cell', format(presynCell,'2d'), 'with permanence', format(permanence,'.2f'))
            connected_synapses = tm.connections.numConnectedSynapses(segment)
            print('cell', format(cell,'2d'), 'segment', format(segment,'2d'), 'has', connected_synapses, 'connected synapse(s)')

def process(cycleArray):
    step = 1
    for sensorValue in cycleArray:
        sensorValueBits = inputSDR.dense
        sensorValueBits = np.zeros(columns)
        sensorValueBits[sensorValue] = 1
        inputSDR.dense = sensorValueBits
        tm.compute(inputSDR, learn = True)
        activeCells = tm.getActiveCells()
        tm.activateDendrites(True)
        activeCellsDense = activeCells.dense
        winnerCellsDense = tm.getWinnerCells().dense
        predictedCellsDense = tm.getPredictiveCells().dense
        utils.cls()
        printHeader(step, sensorValue)
        for rowIdx in range(cellsPerColumn):
            rowData = activeCellsDense[:,rowIdx]
            rowStr = '| Cell   | '
            for colI in range(rowData.size):
                cellName = np.ravel_multi_index([colI, rowIdx], (columns, cellsPerColumn))
                stateActive = activeCellsDense[colI,rowIdx]
                stateWinner = winnerCellsDense[colI,rowIdx]
                statePredicted = predictedCellsDense[colI,rowIdx]
                rowStr += formatCell(cellName, stateActive, stateWinner, statePredicted) + ' | ' 
            print(rowStr)
                
        print(tm.connections)
        printConnectionDetails(tm)
        print()
        step = step + 1
        sleep(0.5)

dashMultiplyer = 50
cycleArray = [0, 1, 2, 3, 4, 5, 6, 7, 6, 5, 4, 3, 2, 1]
cycles = 4
columns = 8
inputSDR = SDR( columns )
cellsPerColumn = 1
tm = TM(columnDimensions          = (inputSDR.size,),
        cellsPerColumn            = cellsPerColumn,     # default: 32
        minThreshold              = 1,                  # default: 10
        activationThreshold       = 1,                  # default: 13
        initialPermanence         = 0.4,                # default: 0.21
        connectedPermanence       = 0.5,                # default: 0.5
        permanenceIncrement       = 0.1,                # default: 0.1
        permanenceDecrement       = 0.1,                # default: 0.1 
        predictedSegmentDecrement = 0.0,                # default: 0.0
        maxSegmentsPerCell        = 1,                  # default: 255
        maxSynapsesPerSegment     = 1                   # default: 255
        )

for cycle in range(cycles):
    process(cycleArray)

5 segment 30 has synapse to cell  4 with permanence 0.40
cell  5 segment 30 has 0 connected synapse(s)
cell  6 segment 32 has synapse to cell  7 with permanence 0.40
cell  6 segment 32 has 0 connected synapse(s)
cell  7 segment  6 has synapse to cell  6 with permanence 0.60
cell  7 segment  6 has 1 connected synapse(s)

--------------------------------------------------
| Cycle  3 | Step  10 | Value   5 | Anomaly: 1.0 |
--------------------------------------------------
| Column |  0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 | 
--------------------------------------------------
| Cell   |  0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 | 
Connections:
    Inputs (6) ~> Outputs (8) via Segments (8)
    Segments on Cell Min/Mean/Max 1 / 1 / 1
    Potential Synapses on Segment Min/Mean/Max 1 / 1 / 1
    Connected Synapses on Segment Min/Mean/Max 0 / 0.25 / 1
    Synapses Dead (0%) Saturated (0%)
    Synapses pruned (0%) Segments pruned (0%)
    Buffer for destroyed synapses: 26    Buffer for destroyed seg